# Task 2

In [464]:
import pandas as pd
import numpy as np
import copy
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations

In [518]:
def support(df, dic, num_of_iter, min_sup, current_dic):
    if dict_depth(dic) == 1:
        for i in df.columns:
            sup = df[df[i] == True].count()[0] / df.shape[0]
            if sup < min_sup:
                df = df.drop(i,axis = 1)
            else:
                dic[i] = {'sup': sup}

        return support(df, dic, 2, min_sup, None)
    
    else:
        if current_dic == None:
            if len(max(df.columns,key=len)) == 1:
                col = ''.join(df.columns)
            else:
                col = df.columns
            lst = []
            new_dic = copy.deepcopy(dic)
            for comb in combinations(col, num_of_iter):
                comb_lst = list(comb)
                ante_sup = df[comb_lst].all(axis=1).value_counts()
                if True in ante_sup:
                    sup = ante_sup[True] / df.shape[0]
                else:
                    sup = 0
                if sup >= min_sup:
                    lst.extend(comb_lst)
                    new_dic[comb] = {'sup': sup}
                    key = ''.join(comb_lst[:num_of_iter-1])
                    dic[key][comb] = {'sup': sup}
            new_col = np.unique(lst)
            df = df[new_col]
            return support(df, dic, num_of_iter + 1, min_sup, new_dic)
                
        else:
            if len(max(df.columns,key=len)) == 1:
                col = ''.join(df.columns)
            else:
                col = df.columns
            lst = []
            new_dic = {}
            for comb in combinations(col, num_of_iter):
                if len(max(comb,key=len)) == 1:
                    sub_col = ''.join(comb)
                else:
                    sub_col = comb
                n = 0
                p = 0
                for sub_comb in combinations(sub_col, num_of_iter - 1):
                    p += 1
                    if sub_comb in current_dic:
                        n += 1
                if n == p:
                    comb_lst = list(comb)
                    ante_sup = df[comb_lst].all(axis=1).value_counts()
                    if True in ante_sup:
                        sup = ante_sup[True] / df.shape[0]
                    else:
                        sup = 0
                    if sup >= min_sup:
                        lst.extend(comb_lst)
                        new_dic[comb] = {'sup': sup}
                        current_dic[comb] = {'sup': sup}
                        for i in range(1,len(comb_lst)):
                            if i == 1:
                                key = ''.join(comb_lst[:i])
                                temp = dic[key]
                            else:
                                key = tuple(comb_lst[:i])
                                temp = temp[key]
                        temp[comb] = {'sup': sup}
            if new_dic:
                new_col = np.unique(lst)
                df = df[new_col]
                return support(df, dic, num_of_iter + 1, min_sup, current_dic)
            
            return dic,current_dic
                

def confidence_and_lift(df, dic, min_sup, min_conf, transaction_dic):
    nest_dic, unnest_dic = support(df, dic, 1, min_sup, None)
    for superset in unnest_dic:
        if len(superset) != 1 and isinstance(superset, tuple):
            sup_superset = unnest_dic[superset]['sup']
            for n in range(1, len(superset)):
                for comb in combinations(superset,n):
                    n_total = 0
                    if n == 1:
                        comb = ''.join(comb)
                        sup_subset = unnest_dic[comb]['sup']
                    else:
                        sup_subset = unnest_dic[comb]['sup']
                    conf = sup_superset / sup_subset
                    if conf >= min_conf:
                        unnest_dic[comb]['confidence'] = conf
                        if isinstance(comb, str):
                            cons = set(superset).difference({comb})
                        else:
                            cons = set(superset).difference(set(comb))
                        if len(cons) == 1:
                            cons = ''.join(cons)
                        else:
                            cons = sorted(cons)
                            cons = tuple(cons)
                        lift = conf / unnest_dic[cons]['sup']
                        if isinstance(comb, str):
                            if isinstance(cons, str):
                                key_name = f'{comb} -> {cons}'
                                n_total = 2
                            else:
                                key_name = f'{comb} -> {", ".join(cons)}'
                                n_total = len(cons) + 1
                        else:
                            if isinstance(cons, str):
                                key_name = f'{", ".join(comb)} -> {cons}'
                                n_total = len(comb) + 1
                            else:
                                key_name = f'{", ".join(comb)} -> {", ".join(cons)}'
                                n_total = len(comb) + len(cons)
                                
                        unnest_dic[comb][key_name] = lift
                        transaction_dic['Association'].append(key_name)
                        transaction_dic['Itemset'].append(n_total)
                        transaction_dic['Lift'].append(lift)
                        transaction_dic['Confidence'].append(conf)
                        transaction_dic['Support'].append(sup_superset)
                    
    return unnest_dic,transaction_dic    


def summary(df, dic, min_sup, min_conf, transaction_dic):
    dic, transaction_dic = confidence_and_lift(df, dic, min_sup, min_conf, transaction_dic)
    transaction_df = pd.DataFrame(transaction_dic)
    transaction_df.sort_values(['Itemset', 'Lift', 'Confidence', 'Support'], 
                               ascending = [True, False, False, False], inplace = True)
    transaction_df.reset_index(drop = True, inplace = True)
    return transaction_df


def dict_depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(dict_depth, d.values())) if d else 0)
    return 0


# task 3

In [522]:
dataset = [['A', 'B', 'C', 'D', 'F', 'G', 'H'],
           ['A', 'B', 'C', 'D'],
           ['A', 'B', 'C', 'D'],
           ['A', 'B', 'D'],
           ['B', 'C', 'E']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)
min_sup = .25
min_conf = .8
dic = {}
transaction_dic = {'Association':[], 'Lift':[], 'Confidence':[], 'Support':[], 'Itemset':[]}

summary(df, dic, min_sup, min_conf, transaction_dic)

,Association,Lift,Confidence,Support,Itemset
0,A -> D,1.25,1.0,0.8,2
1,D -> A,1.25,1.0,0.8,2
2,A -> B,1.00,1.0,0.8,2
3,C -> B,1.00,1.0,0.8,2
4,D -> B,1.00,1.0,0.8,2
5,B -> A,1.00,0.8,0.8,2
6,B -> C,1.00,0.8,0.8,2
7,B -> D,1.00,0.8,0.8,2
8,"A -> B, D",1.25,1.0,0.8,3
9,"D -> A, B",1.25,1.0,0.8,3


# task 4

In [520]:
data = pd.read_csv('data/supermarket.csv', names = range(49))
data.fillna('', inplace=True)
dataset = data.values
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

df = pd.DataFrame(te_ary, columns=te.columns_)
df.drop(columns = '', inplace = True)
min_sup = .15
min_conf = .8
dic = {}
transaction_dic = {'Association':[], 'Lift':[], 'Confidence':[],'Support':[], 'Itemset':[]}

summary(df, dic, min_sup, min_conf, transaction_dic)

,Association,Lift,Confidence,Support,Itemset
0,total = high -> bread and cake,1.169356,0.841572,0.305381,2
1,small goods -> bread and cake,1.160398,0.835125,0.201426,2
2,canned fruit -> bread and cake,1.126320,0.810600,0.224768,2
3,jams-spreads -> bread and cake,1.116593,0.803599,0.221958,2
4,canned fish-meat -> bread and cake,1.113364,0.801275,0.162957,2
...,...,...,...,...,...
950,"baking needs, frozen foods, juice-sat-cord-ms, vegetables -> bread and cake",1.176979,0.847059,0.155608,5
951,"baking needs, biscuits, frozen foods, sauces-gravy-pkle -> bread and cake",1.176367,0.846618,0.151502,5
952,"baking needs, biscuits, bread and cake, frozen foods, vegetables -> fruit",1.318960,0.844340,0.154744,6
953,"baking needs, biscuits, bread and cake, frozen foods, fruit -> vegetables",1.310136,0.838407,0.154744,6
